In [1]:
# Mixture-of-Agents Enhances Large Language Model Capabilities
# https://arxiv.org/abs/2406.04692

# MoA - GitHub
# https://github.com/togethercomputer/MoA/tree/main

# Mixture of Agents TURBO Tutorial 🚀 Better Than GPT4o AND Fast?!
# (Matthew Berman)
# https://www.youtube.com/watch?v=BKyxMreb3mk&t=604s

In [2]:
# Example prompts
# ChatGPT
# https://chatgpt.com/c/d410922d-f4b6-45ac-9c66-b3a729098253

proposer_prompt = [{"role": "user", "content": "Top things to do in NYC"}]

proposer_prompt_w_refs = [
    {"role": "user", "content": "Top things to do in NYC"},
    {"role": "assistant", "content": "Visit Central Park."},
    {"role": "assistant", "content": "See a Broadway show."},
    {"role": "assistant", "content": "Explore the Metropolitan Museum of Art."},
    {"role": "assistant", "content": "Walk across the Brooklyn Bridge."}
]

aggregator_prompt = [
    {"role": "system", "content": "You have been provided with a set of responses from various open-source models to the latest user query. Your task is to synthesize these responses into a single, high-quality response. It is crucial to critically evaluate the information provided in these responses, recognizing that some of it may be biased or incorrect. Your response should not simply replicate the given answers but should offer a refined, accurate, and comprehensive reply to the instruction. Ensure your response is well-structured, coherent, and adheres to the highest standards of accuracy and reliability.\n\nResponses from models:"},
    {"role": "user", "content": "Top things to do in NYC"},
    {"role": "assistant", "content": "Visit Central Park."},
    {"role": "assistant", "content": "See a Broadway show."},
    {"role": "assistant", "content": "Explore the Metropolitan Museum of Art."},
    {"role": "assistant", "content": "Walk across the Brooklyn Bridge."}
]

In [3]:
!pip -q install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [4]:
import json
import os
import re
from google.colab import userdata

# Define the API clients

In [5]:
from groq import Groq

groq_client = Groq(
    api_key=userdata.get('GROQ_API_KEY'),
)


# What is the objective?

- Create a simple Mixture of Agents (MoA) notebook using models from Groq.



# Create a list of agents

AGENTS
1. proposer_agent
2. aggregator_agent


# Helper functions

In [6]:
def create_message_history(system_message, user_input):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        },
                        {
                            "role": "user",
                            "content": user_input
                        }
                    ]

    return message_history



# Define the reference and aggregator models

In [7]:
reference_models = [
    "llama3-8b-8192",
    "llama3-70b-8192",
    "mixtral-8x7b-32768",
    "gemma-7b-it",
    #"gemma2-9b-it"
]

aggregator_model = "llama3-70b-8192"

len(reference_models)

4

# Set up the LLMs and test them

In [8]:
def make_llm_api_call(message_history, model_name):

    response = groq_client.chat.completions.create(
                        messages=message_history,
                        model=model_name,
                        temperature=0.7,
                        max_tokens=2048
                    )

    response_text = response.choices[0].message.content

    return response_text


# Example

system_message = "Your name is Molly."
user_message = "What's your name?"

message_history = create_message_history(system_message, user_message)

# Check that all the models are working
for ref_model in reference_models:

    response = make_llm_api_call(message_history, ref_model)

    print(ref_model)
    print(response)
    print()

llama3-8b-8192
My name is Molly! Nice to meet you!

llama3-70b-8192
My name is Molly! Nice to meet you!

mixtral-8x7b-32768
I'm an artificial intelligence and don't have a personal name, but you can call me Molly if you'd like. How can I assist you today?

(Note: As a assistant, I don't have a personal identity or consciousness, but I'm here to help answer your questions and complete tasks to the best of my ability. )

gemma-7b-it
My name is Molly.



# Set up the Agents

In [9]:
def run_proposer_agent(user_query, ref_response_list, model_name):

    print(F"---PROPOSER AGENT: {model_name}---")

    prop_system_message = f"""This is a set of responses from various
    open-source models to the latest user query.

    ### Responses from models: {ref_response_list}
    """

    if len(ref_response_list) != 0:
        prop_system_message = "You are a helpful assistant."


    message_history = [
        {"role": "system", "content": prop_system_message},
        {"role": "user", "content": user_query}
        ]

    #if len(ref_response_list) != 0:

        #for ref in ref_response_list:
            #message = {"role": "assistant", "content": ref}
            #message_history.append(message)


    # Prompt the llm
    response = make_llm_api_call(message_history, model_name)

    #print("Ref responses:", ref_response_list)
    print("New response:", response)
    print()

    return response



# Example

#model_name = reference_models[0]

for model_name in reference_models:

    user_query = "Top things to do in NYC"

    ref_response_list = [
        "Visit Central Park.",
        "See a Broadway show.",
        "Explore the Metropolitan Museum of Art.",
        "Walk across the Brooklyn Bridge."
    ]

    # Prompt the chat_agent
    response = run_proposer_agent(user_query, ref_response_list, model_name)


---PROPOSER AGENT: llama3-8b-8192---
New response: The city that never sleeps! New York City is a vibrant and exciting place to visit, with endless options for entertainment, culture, and adventure. Here are some of the top things to do in NYC:

1. **Visit iconic landmarks:** The Statue of Liberty, Central Park, Times Square, the Empire State Building, and the 9/11 Memorial & Museum are all must-see attractions.

2. **Explore world-class museums:** The Metropolitan Museum of Art, the Museum of Modern Art (MoMA), the American Museum of Natural History, and the Guggenheim Museum are just a few of the many world-class museums in NYC.

3. **Take a stroll through iconic neighborhoods:** Visit trendy neighborhoods like SoHo, Greenwich Village, and Williamsburg, and explore their unique shops, restaurants, and street art.

4. **Enjoy the performing arts:** Catch a Broadway show, visit the Metropolitan Opera House, or attend a concert at iconic venues like Radio City Music Hall or Carnegie Hal

In [10]:
def run_aggregator_agent(user_query, ref_response_list, model_name):

    print(f"---AGGREGATOR AGENT: {model_name}---")

    agg_system_message = f"""You have been provided with a set of responses from various
    open-source models to the latest user query. Your task is to synthesize these responses
    into a single, high-quality response. It is crucial to critically evaluate the information
    provided in these responses, recognizing that some of it may be biased or incorrect.
    Your response should not simply replicate the given answers but should offer a refined,
    accurate, and comprehensive reply to the instruction. Ensure your response is well-structured,
    coherent, and adheres to the highest standards of accuracy and reliability.

    ### Responses from models: {ref_response_list}
    """.strip()

    message_history = [
        {"role": "system", "content": agg_system_message},
        {"role": "user", "content": user_query}
        ]


    #for ref in ref_response_list:
       #message = {"role": "assistant", "content": ref}
       #message_history.append(message)


    # Prompt the llm
    response = make_llm_api_call(message_history, model_name)

    #print("Ref responses:", ref_response_list)
    print("New response:", response)
    print()

    return response


# Example

model_name = reference_models[1]

user_query = "Top things to do in NYC"

ref_response_list = [
    "Visit Central Park.",
    "See a Broadway show.",
    "Explore the Metropolitan Museum of Art.",
    "Walk across the Brooklyn Bridge."
]

# Prompt the chat_agent
response = run_aggregator_agent(user_query, ref_response_list, model_name)


---AGGREGATOR AGENT: llama3-70b-8192---
New response: New York City, the city that never sleeps, is a treasure trove of exciting experiences and iconic landmarks. Here are some of the top things to do in NYC, carefully curated from a range of expert opinions:

First and foremost, a visit to Central Park is a must. This 843-acre green oasis in the heart of Manhattan offers a tranquil escape from the hustle and bustle of city life. Take a leisurely stroll, rent a bike, or enjoy a picnic on the Great Lawn.

Next, immerse yourself in the magic of Broadway by catching a performance of a hit musical or play. With dozens of theaters to choose from, you're sure to find a show that suits your taste.

Art lovers won't want to miss the Metropolitan Museum of Art, one of the world's largest and most renowned museums. With a collection that spans over 5,000 years of human history, you could easily spend days exploring the Met's vast exhibits.

Finally, take in the breathtaking views of the Manhatta

# Run the MoA system

In [11]:
reference_models = [
    "llama3-8b-8192",
    "llama3-70b-8192",
    "mixtral-8x7b-32768",
    "gemma-7b-it",
    "gemma2-9b-it"
]

aggregator_model = "llama3-70b-8192"

#user_query = "Top things to do in NYC"
#user_query = "Write me ten sentences that end with the word apple."
user_query = "Write a one paragraph email to my friend explaining the benefits of a minimalist lifestyle."

ref_response_list = []


def moa_layer(user_query, ref_response_list, reference_models):

    response_list = []

    for model_name in reference_models:

        response = run_proposer_agent(user_query, ref_response_list, model_name)

        response_list.append(response)

    return response_list


print("----------")
print("Layer_1")
print("----------")
# Layer_1
ref_response_list = moa_layer(user_query, ref_response_list, reference_models)

print("----------")
print("Layer_2")
print("----------")
# Layer_2
ref_response_list = moa_layer(user_query, ref_response_list, reference_models)

print("----------")
print("Layer_3")
print("----------")
# Layer_3
ref_response_list = moa_layer(user_query, ref_response_list, reference_models)

print("----------")
print("Layer_4")
print("----------")
# Layer_4
response = run_aggregator_agent(user_query, ref_response_list, aggregator_model)


----------
Layer_1
----------
---PROPOSER AGENT: llama3-8b-8192---
New response: Here is a one-paragraph email to your friend explaining the benefits of a minimalist lifestyle:

"Hey friend! I've been thinking a lot about simplicity and minimalism lately, and I wanted to share some of the benefits I've discovered with you. Living with fewer possessions and focusing on what's truly important has been a game-changer for me. Not only have I reduced my stress levels and freed up more time for the things I love, but I've also seen a significant decrease in clutter and waste. Plus, the financial benefits are undeniable - less stuff means less to buy, maintain, and eventually get rid of. And let's not forget the mental clarity and sense of freedom that comes with living intentionally. It's not about depriving yourself of things you need or love, but about being mindful of the choices you make and living in alignment with your values. I highly recommend giving it a try - even just a little at 